# Train Model with CPU

In [ ]:
import tensorflow as tf
from tensorflow.python.client import timeline
import pylab
import numpy as np
import os

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

tf.logging.set_verbosity(tf.logging.INFO)

## Reset TensorFlow Graph
Useful in Jupyter Notebooks

In [ ]:
tf.reset_default_graph()

## Create TensorFlow Session

In [ ]:
sess = tf.Session()
print(sess)

### Load Model Training and Test/Validation Data 


In [ ]:
num_samples = 100000

In [ ]:
import numpy as np

x_train = np.random.rand(num_samples).astype(np.float32)
print(x_train)

noise = np.random.normal(scale=0.01, size=len(x_train))

y_train = x_train * 0.1 + 0.3 + noise
print(y_train)

pylab.plot(x_train, y_train, '.')

In [ ]:
import pylab

x_test = np.random.rand(len(x_train)).astype(np.float32)
print(x_test)

noise = np.random.normal(scale=.01, size=len(x_train))

y_test = x_test * 0.1 + 0.3 + noise
print(y_test)

pylab.plot(x_test, y_test, '.')

In [ ]:
with tf.device("/cpu:0"):
    W = tf.get_variable(shape=[], name='weights')
    print(W)

    b = tf.get_variable(shape=[], name='bias')
    print(b)

    x_observed = tf.placeholder(shape=[None], 
                                dtype=tf.float32, 
                                name='x_observed')
    print(x_observed)

    y_pred = W * x_observed + b
    print(y_pred)

In [ ]:
learning_rate = 0.025

with tf.device("/cpu:0"):
    y_observed = tf.placeholder(shape=[None], dtype=tf.float32, name='y_observed')
    print(y_observed)

    loss_op = tf.reduce_mean(tf.square(y_pred - y_observed))
    optimizer_op = tf.train.GradientDescentOptimizer(learning_rate)
    
    train_op = optimizer_op.minimize(loss_op)  

    print("Loss Scalar: ", loss_op)
    print("Optimizer Op: ", optimizer_op)
    print("Train Op: ", train_op)

## Randomly Initialize Variables (Weights and Bias)
The goal is to learn more accurate Weights and Bias during training.

In [ ]:
with tf.device("/cpu:0"):
    init_op = tf.global_variables_initializer()
    print(init_op)

In [ ]:
sess.run(init_op)
print("Initial random W: %f" % sess.run(W))
print("Initial random b: %f" % sess.run(b))

## View Accuracy of Pre-Training, Initial Random Variables
We want this to be close to 0, but it's relatively far away.  This is why we train!

In [ ]:
def test(x, y):
    return sess.run(loss_op, feed_dict={x_observed: x, y_observed: y})

In [ ]:
test(x_test, y_test)

## Setup Loss Summary Operations for Tensorboard

In [ ]:
loss_summary_scalar_op = tf.summary.scalar('loss', loss_op)
loss_summary_merge_all_op = tf.summary.merge_all()

In [ ]:
train_summary_writer = tf.summary.FileWriter('./linear_model/logs/cpu/train/', 
                                            graph=tf.get_default_graph())

test_summary_writer = tf.summary.FileWriter('./linear_model/logs/cpu/test/',
                                            graph=tf.get_default_graph())

## Train Model

In [ ]:
%%time

from tensorflow.python.client import timeline

with tf.device("/cpu:0"):
    run_metadata = tf.RunMetadata()
    max_steps = 401
    for step in range(max_steps):
        if (step < max_steps - 1):
            test_summary_log, _ = sess.run([loss_summary_merge_all_op, loss_op], feed_dict={x_observed: x_test, y_observed: y_test})
            train_summary_log, _ = sess.run([loss_summary_merge_all_op, train_op], feed_dict={x_observed: x_train, y_observed: y_train})
        else:  
            test_summary_log, _ = sess.run([loss_summary_merge_all_op, loss_op], feed_dict={x_observed: x_test, y_observed: y_test})
            train_summary_log, _ = sess.run([loss_summary_merge_all_op, train_op], feed_dict={x_observed: x_train, y_observed: y_train}, 
                                            options=tf.RunOptions(trace_level=tf.RunOptions.SOFTWARE_TRACE), 
                                            run_metadata=run_metadata)

            trace = timeline.Timeline(step_stats=run_metadata.step_stats)    
            with open('timeline-cpu.json', 'w') as trace_file:
                trace_file.write(trace.generate_chrome_trace_format(show_memory=True))

        if step % 10 == 0:
            print(step, sess.run([W, b]))
            train_summary_writer.add_summary(train_summary_log, step)
            train_summary_writer.flush()
            test_summary_writer.add_summary(test_summary_log, step)
            test_summary_writer.flush()

In [ ]:
pylab.plot(x_train, y_train, '.', label="target")
pylab.plot(x_train, sess.run(y_pred, 
                             feed_dict={x_observed: x_train, 
                                        y_observed: y_train}), 
           ".", 
           label="predicted")
pylab.legend()
pylab.ylim(0, 1.0)

## View Loss Summaries in Tensorboard
Navigate to the **`Scalars`** and **`Graphs`** tab in TensorBoard

## Save Graph For Optimization
We will use this later.

In [ ]:
import os
checkpoint_base_path = './linear_model/cpu/checkpoint'

saver = tf.train.Saver()

graph_model_path = '%s/graph.pb' % checkpoint_base_path
print(graph_model_path)

os.makedirs(checkpoint_base_path, exist_ok=True)

tf.train.write_graph(sess.graph_def, 
                     '.', 
                     graph_model_path,
                     as_text=False) 


checkpoint_model_path = '%s/model.ckpt' % checkpoint_base_path
saver.save(sess, 
           save_path=checkpoint_model_path)

print(checkpoint_model_path)

In [ ]:
os.listdir(checkpoint_base_path)

In [ ]:
sess.close()

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
from google.protobuf import text_format
from tensorflow.core.framework import graph_pb2

def convert_graph_to_dot(input_graph, output_dot, is_input_graph_binary):
    graph = graph_pb2.GraphDef()
    with open(input_graph, "rb") as fh:
        if is_input_graph_binary:
            graph.ParseFromString(fh.read())
        else:
            text_format.Merge(fh.read(), graph)
    with open(output_dot, "wt") as fh:
        print("digraph graphname {", file=fh)
        for node in graph.node:
            output_name = node.name
            print("  \"" + output_name + "\" [label=\"" + node.op + "\"];", file=fh)
            for input_full_name in node.input:
                parts = input_full_name.split(":")
                input_name = re.sub(r"^\^", "", parts[0])
                print("  \"" + input_name + "\" -> \"" + output_name + "\";", file=fh)
        print("}", file=fh)
        print("Created dot file '%s' for graph '%s'." % (output_dot, input_graph))

In [ ]:
output_dot_path='./graph_cpu.dot'
convert_graph_to_dot(input_graph=graph_model_path, output_dot=output_dot_path, is_input_graph_binary=True)

In [ ]:
%%bash -s "$output_dot_path"

dot -T png $1 \
    -o ./graph_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('./graph_cpu.png', width=1024, height=768)

# Prepare Optimized Model for Deployment

## Freeze Fully Optimized Graph

In [ ]:
from tensorflow.python.tools import freeze_graph

model_parent_path = './linear_model/cpu/checkpoint'

model_graph_path = '%s/graph.pb' % model_parent_path
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
model_checkpoint_path = '%s/model.ckpt' % model_parent_path

freeze_graph.freeze_graph(input_graph=model_graph_path, 
                          input_saver="",
                          input_binary=True, 
                          input_checkpoint=model_checkpoint_path,
                          output_node_names="add",
                          restore_op_name="save/restore_all", 
                          filename_tensor_name="save/Const:0",
                          output_graph=frozen_model_graph_path, 
                          clear_devices=True, 
                          initializer_nodes="")
print(frozen_model_graph_path)

In [ ]:
%%bash

ls -l ./linear_model/cpu/checkpoint/

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import re
from google.protobuf import text_format
from tensorflow.core.framework import graph_pb2

def convert_graph_to_dot(input_graph, output_dot, is_input_graph_binary):
    graph = graph_pb2.GraphDef()
    with open(input_graph, "rb") as fh:
        if is_input_graph_binary:
            graph.ParseFromString(fh.read())
        else:
            text_format.Merge(fh.read(), graph)
    with open(output_dot, "wt") as fh:
        print("digraph graphname {", file=fh)
        for node in graph.node:
            output_name = node.name
            print("  \"" + output_name + "\" [label=\"" + node.op + "\"];", file=fh)
            for input_full_name in node.input:
                parts = input_full_name.split(":")
                input_name = re.sub(r"^\^", "", parts[0])
                print("  \"" + input_name + "\" -> \"" + output_name + "\";", file=fh)
        print("}", file=fh)
        print("Created dot file '%s' for graph '%s'." % (output_dot, input_graph))
        

In [ ]:
input_graph='./linear_model/cpu/checkpoint/graph.pb'
output_dot='./frozen_model_graph_cpu.dot'
convert_graph_to_dot(input_graph=input_graph, output_dot=output_dot, is_input_graph_binary=True)

In [ ]:
%%bash

dot -T png ./frozen_model_graph_cpu.dot \
    -o ./frozen_model_graph_cpu.png > /tmp/a.out

In [ ]:
from IPython.display import Image

Image('./frozen_model_graph_cpu.png')

# Save Model for Deployment and Inference

## Reset Default Graph

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

## Create New Session

In [ ]:
sess = tf.Session()

## Load Frozen Graph

In [ ]:
from tensorflow.python.tools import inspect_checkpoint

inspect_checkpoint.print_tensors_in_checkpoint_file(file_name="./linear_model/cpu/checkpoint/model.ckpt",
                                                    tensor_name="",
                                                    all_tensors=True,
                                                    all_tensor_names=True)

In [ ]:
saver = tf.train.import_meta_graph('./linear_model/cpu/checkpoint/model.ckpt.meta')
saver.restore(sess, './linear_model/cpu/checkpoint/model.ckpt')

model_parent_path = './linear_model/cpu/checkpoint/'
frozen_model_graph_path = '%s/frozen_model_graph_cpu.pb' % model_parent_path
print(frozen_model_graph_path)

with tf.gfile.GFile(frozen_model_graph_path, 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

tf.import_graph_def(
    graph_def, 
    input_map=None, 
    return_elements=None, 
    name="", 
    op_dict=None, 
    producer_op_list=None
)

print("weights = ", sess.run("weights:0"))
print("bias = ", sess.run("bias:0"))

## Create `SignatureDef` Asset for TensorFlow Serving

In [ ]:
from tensorflow.python.saved_model import utils
from tensorflow.python.saved_model import signature_constants
from tensorflow.python.saved_model import signature_def_utils

graph = tf.get_default_graph()

x_observed = graph.get_tensor_by_name('x_observed:0')
y_pred = graph.get_tensor_by_name('add:0')

inputs_map = {'inputs': x_observed}
outputs_map = {'outputs': y_pred}

predict_signature = signature_def_utils.predict_signature_def(
                inputs = inputs_map, 
                outputs = outputs_map)
print(predict_signature)

## Save Model with Assets

In [ ]:
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants
from shutil import rmtree
from tensorflow.contrib import predictor
import numpy as np
import os

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'

os.makedirs(saved_model_path, exist_ok=True)
rmtree(saved_model_path)

import tensorflow.saved_model as saved_model
from shutil import rmtree

saved_model.simple_save(sess,
            saved_model_path,
            inputs={'inputs': x_observed},
            outputs={"outputs": y_pred})

In [ ]:
import os
print(saved_model_path)
os.listdir(saved_model_path)
os.listdir('%s/variables' % saved_model_path)

In [ ]:
%%bash
echo "./linear_model/cpu/pipeline_tfserving/0"
echo ""
ls -al ./linear_model/cpu/pipeline_tfserving/0

## Inspect with [Saved Model CLI](https://www.tensorflow.org/guide/saved_model)
Note:  This takes a minute or two for some reason.  Please be patient.

In [ ]:
import subprocess

output = subprocess.run(["saved_model_cli", "show", \
                "--dir", saved_model_path, "--all"], \
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE)

print(output.stdout.decode('utf-8'))

# Optimized Model for Deployment

## Predict with Python (SLOW)


In [ ]:
from tensorflow.contrib import predictor
import numpy as np

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'
print(saved_model_path)

input_shape = 1
input_data = np.random.random_sample(input_shape)

predict_fn = predictor.from_saved_model(saved_model_path)

In [ ]:
%%time
predictions = predict_fn({'inputs': input_data})

print('Prediction: %s' % predictions["outputs"])

## Optimize with TensorFlow Lite

![PipelineAI + TensorFlow Lite](https://raw.githubusercontent.com/PipelineAI/site/master/assets/img/toco-optimizer.png)

In [ ]:
import tensorflow as tf
import os

from shutil import rmtree

saved_model_path = './linear_model/cpu/pipeline_tfserving/0'
print(saved_model_path)

tflite_model_base_path = './linear_model/cpu/tflite/'

os.makedirs(tflite_model_base_path, exist_ok=True)

converter = tf.lite.TocoConverter.from_saved_model(saved_model_path)

# TF 1.11+
converter.post_training_quantize = True

tflite_model = converter.convert()

tflite_model_path = '%s/tflite_optimized_model.tflite' % tflite_model_base_path

model_size = open(tflite_model_path, "wb").write(tflite_model)

print('\nModel size reduced to %s bytes' % model_size)

In [ ]:
%%bash -s "$tflite_model_path"
echo "ls -al $1"
echo ""
ls -al $1

In [ ]:
import numpy as np
import tensorflow as tf

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

In [ ]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
print('Input Tensor Details: %s' % input_details)

output_details = interpreter.get_output_details()
print('Output Tensor Details: %s' % output_details)

In [ ]:
# Test model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
print('Input: %s' % input_data)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [ ]:
%%time
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(output_details[0]['index'])
print('Prediction: %s' % output_data)